In [1]:
import pandas as pd
import numpy as np
from pandas import Series, DataFrame
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
from matplotlib import pyplot as plt
import pickle
import import_ipynb
import sys
sys.path.append('./../')
import Portfolio_value as pv

In [2]:
df =  pd.read_csv('./../data/data.csv')

In [3]:
p_val = np.load('./../result/p_val/p_val_dyna.npy', allow_pickle=True)
w_val = np.load('./../result/w_val/w_val_dyna.npy', allow_pickle=True)

p_val = p_val.reshape(1)
p_val = p_val[0]

w_val = w_val.reshape(1)
w_val = w_val[0]

In [4]:

# for upper_year in [15,20,25]:
#     for k in [0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0]:
#         for withdraw_rate in [0.04,0.06,0.08,0.10]:
#             for year in range(1980,2003-upper_year):
             

In [5]:
normal_year = np.load('./../result/year/normal_year.npy', allow_pickle=True).reshape(1)[0]
bear_year = np.load('./../result/year/bear_year.npy', allow_pickle=True).reshape(1)[0]
full_year = np.load('./../result/year/full_year.npy', allow_pickle=True).reshape(1)[0]

In [6]:
upper_years = [20, 25, 30]

In [7]:
normal_perform = {}
bear_perform = {}
full_perform = {}
for upper_year in upper_years:
    normal_perform[upper_year] = {
    'dep':[],
    'max':[],
    'under':[],
    'inc':[],
    'beq':[],
    'total':[]
}
    bear_perform[upper_year] = {
    'dep':[],
    'max':[],
    'under':[],
    'inc':[],
    'beq':[],
    'total':[]
}
    
    full_perform[upper_year] = {
    'dep':[],
    'max':[],
    'under':[],
    'inc':[],
    'beq':[],
    'total':[]
}

In [8]:
init_wealth = 350000
k = 0.5
withdraw_rate = 0.08

for upper_year in upper_years:
    for years in normal_year[upper_year] :
        for year in years:
            res = pv.get_metric(year, k, upper_year, withdraw_rate, df, init_wealth, p_val, w_val)
            if res[2] < upper_year*12 : #고갈 시점이 upper_year보다 작으면 (이미 고갈이 되면)
                normal_perform[upper_year]['dep'].append(100)
            elif res[2] >= upper_year*12: # 고갈이 안되면
                normal_perform[upper_year]['dep'].append(0)
            normal_perform[upper_year]['max'].append(res[3])
            normal_perform[upper_year]['under'].append(res[4])
            normal_perform[upper_year]['inc'].append(res[1])
            normal_perform[upper_year]['beq'].append(res[0])
            normal_perform[upper_year]['total'].append(res[1] + res[0])

    for years in bear_year[upper_year]:
        for year in years:
            res = pv.get_metric(year, k, upper_year, withdraw_rate, df, init_wealth, p_val, w_val)
            if res[2] < upper_year*12 : #고갈 시점이 upper_year보다 작으면 (이미 고갈이 되면)
                bear_perform[upper_year]['dep'].append(100)
            elif res[2] >= upper_year*12: # 고갈이 안되면
                bear_perform[upper_year]['dep'].append(0)
            bear_perform[upper_year]['max'].append(res[3])
            bear_perform[upper_year]['under'].append(res[4])
            bear_perform[upper_year]['inc'].append(res[1])
            bear_perform[upper_year]['beq'].append(res[0])
            bear_perform[upper_year]['total'].append(res[1] + res[0])

    for years in full_year[upper_year]:
        for year in years:
            res = pv.get_metric(year, k, upper_year, withdraw_rate, df, init_wealth, p_val, w_val)
            if res[2] < upper_year*12 : #고갈 시점이 upper_year보다 작으면 (이미 고갈이 되면)
                full_perform[upper_year]['dep'].append(100)
            elif res[2] >= upper_year*12: # 고갈이 안되면
                full_perform[upper_year]['dep'].append(0)
            full_perform[upper_year]['max'].append(res[3])
            full_perform[upper_year]['under'].append(res[4])
            full_perform[upper_year]['inc'].append(res[1])
            full_perform[upper_year]['beq'].append(res[0])
            full_perform[upper_year]['total'].append(res[1] + res[0])


In [9]:
# Define all dictionaries and indices we want to calculate mean for
dicts = [normal_perform, bear_perform, full_perform]
indices = upper_years

# Use list comprehension to compute all means at once
# Store results in lists
a = [np.round(np.mean(d[i]['dep']),1) for i in indices for d in dicts]
b = [np.round(np.mean(d[i]['max']),1) for i in indices for d in dicts]
c = [np.round(np.mean(d[i]['under']),1) for i in indices for d in dicts]
d = [np.round(np.mean(d[i]['inc']),1) for i in indices for d in dicts]
e = [np.round(np.mean(d[i]['beq']),1) for i in indices for d in dicts]
f = [np.round(np.mean(d[i]['total']),1) for i in indices for d in dicts]

# If you need to access the results separately, you can do so by indexing the lists:
# For example, a1 corresponds to a_results[0], a2 corresponds to a_results[1], etc.

In [10]:

print('Depletion Prob. (\%) & & {} & {} & {} & & {} & {} & {} & & {} & {} & {}'.format(a[0], a[1], a[2], a[3], a[4], a[5], a[6], a[7], a[8]))
print('Max. decline (\%) & & {} & {} & {} & & {} & {} & {} & & {} & {} & {}'.format(b[0], b[1], b[2], b[3], b[4], b[5], b[6], b[7], b[8]))
print('Underwater duration (month) & & {} & {} & {} & & {} & {} & {} & & {} & {} & {}'.format(c[0], c[1], c[2], c[3], c[4], c[5], c[6], c[7], c[8]))
print('Total withdrawal (\%) & & {} & {} & {} & & {} & {} & {} & & {} & {} & {}'.format(d[0], d[1], d[2], d[3], d[4], d[5], d[6], d[7], d[8]))
print('Bequest (\%) & & {} & {} & {} & & {} & {} & {} & & {} & {} & {}'.format(e[0], e[1], e[2], e[3], e[4], e[5], e[6], e[7], e[8]))
print('Total withdrawal + Bequest (\%) & & {} & {} & {} & & {} & {} & {} & & {} & {} & {}'.format(f[0], f[1], f[2], f[3], f[4], f[5], f[6], f[7], f[8]))






Depletion Prob. (\%) & & 12.5 & 80.0 & 56.5 & & 50.0 & 100.0 & 77.8 & & 100.0 & 100.0 & 100.0
Max. decline (\%) & & -84.8 & -99.2 & -94.2 & & -96.2 & -100.0 & -98.3 & & -100.0 & -100.0 & -100.0
Underwater duration (month) & & 89.2 & 226.7 & 178.9 & & 149.2 & 286.8 & 225.7 & & 209.2 & 345.4 & 261.6
Total withdrawal (\%) & & 119.6 & 89.9 & 100.2 & & 134.1 & 80.9 & 104.5 & & 139.0 & 82.4 & 117.2
Bequest (\%) & & 16.1 & 0.9 & 6.2 & & 3.9 & 0.0 & 1.7 & & 0.0 & 0.0 & 0.0
Total withdrawal + Bequest (\%) & & 135.7 & 90.8 & 106.4 & & 138.0 & 80.9 & 106.3 & & 139.0 & 82.4 & 117.2


# Result

In [11]:
# 매트릭 참고

# https://papers.ssrn.com/sol3/papers.cfm?abstract_id=2544651
# https://investmentsandwealth.org/getattachment/90eb6376-d090-4904-9f82-786553ff5ed9/RMJ023-OptimalWithdrawalStrategy.pdf
# https://www.betterfinancialeducation.com/sites/larryfrank.us1.advisor.ws/files/documents/apr09_frank_and_blanchett.pdf
# https://www.tandfonline.com/doi/epdf/10.2469/faj.v61.n6.2776?needAccess=true&role=button